# Settings

## Constants

In [1]:
import os

PATH_DATA = './Dataset'
sensor_data = ['UserInfo.csv', 'Service.csv', 'ContextualFactor.csv', 'Availability.csv']

# Utility Functions

In [2]:
# Install required packages
!pip install -q scikit-learn catboost
!pip install -q scikit-learn catboost xgboost lightgbm imbalanced-learn

# === Core Libraries ===
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta

# Set timezone to UTC+9 for consistent time handling
tz = timezone(+timedelta(hours=9))

# === Preprocessing and Utilities ===
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# === Model Selection and Evaluation ===
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, f1_score

# === Machine Learning Models ===
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

# === Handling Imbalanced Data ===
from imblearn.over_sampling import SMOTE


# Load the Dataset into Dataframe

In [3]:
dataframes = {
    filename: pd.read_csv(os.path.join(PATH_DATA, filename)).reset_index(drop=True)
    for filename in sensor_data
}
dfService = dataframes['Service.csv']
dfContextualFactor = dataframes['ContextualFactor.csv']
dfUserInfo = dataframes['UserInfo.csv']
dfAvailability = dataframes['Availability.csv']

# Preprocessing

In [5]:
# Select specific columns from dfContextualFactor, dfService, and dfUserInfo DataFrames
# Combine relevant columns to create a unified dataset
dfContextualFactor_selected_columns=dfContextualFactor[['uid','sid','activity1','activity2','activity3','userRoom','userPosition',]]
dfService_selected_columns=dfService[['weekOfExperiment','dayOfWeek','startTime', 'activityInquiry','availabilityInquiry','speechShadowing','continue-to-nextInquiry_1','continue-to-nextInquiry_2','continue-to-nextInquiry_3','continue-to-nextInquiry_4','endTime','endType']]
dfAvailability_selected_columns=dfAvailability[['SHORT_INTERACTION_availability', 'LONG_INTERACTION_availability']]

## Create binary columns for interaction types
## SHORT_INTERACTION: True if availabilityInquiry is not NaN
# dfCombinedAll['SHORT_INTERACTION_availability'] = dfCombinedAll['availabilityInquiry'].notna()
## LONG_INTERACTION: True if continue-to-nextInquiry_1 is not NaN
# dfCombinedAll['LONG_INTERACTION_availability'] = dfCombinedAll['continue-to-nextInquiry_1'].notna()

# ! For those who want to redefine LONG_INTERACTION with thresholds longer than 3 minutes,
# you can use the following columns:
# 5 minutes  => use continue-to-nextInquiry_2
# 7 minutes  => use continue-to-nextInquiry_3
# 9 minutes  => use continue-to-nextInquiry_4


dfCombinedAll=pd.concat([dfContextualFactor_selected_columns, dfService_selected_columns,dfAvailability_selected_columns], axis=1)

In [7]:
# Concatenate and merge the all columns from dfUserInfo (dfUserInfo.csv)
dfUserInfo_selected_columns = dfUserInfo[['uid', 'homeType', 'speakerRoom', 'speakerPosition']]
dfCombinedAll = pd.merge(dfCombinedAll, dfUserInfo_selected_columns, on='uid', how='left')

## Position Processing

In [10]:
# Define function to calculate proximity between user and speaker
def calculate_proximity(row):
    # Return 0 if user and speaker are in different rooms
    if row['userRoom'] != row['speakerRoom']:
        return 0
    # If in the same room
    elif row['userPosition'] == row['speakerPosition']:
        return 2  # Same position
    else:
        return 1  # Different positions (including missing position)

# Apply proximity calculation to create a new 'proximity' column
dfCombinedAll['proximity'] = dfCombinedAll.apply(calculate_proximity, axis=1)

print(dfCombinedAll[['userRoom', 'userPosition', 'speakerRoom', 'speakerPosition', 'proximity']].head(10))


      userRoom userPosition speakerRoom speakerPosition  proximity
0     Bed Room          Bed    Bed Room            Desk          1
1    Rest Room          NaN    Bed Room            Desk          0
2  Living Room          NaN    Bed Room            Desk          0
3  Living Room          NaN    Bed Room            Desk          0
4     Bed Room          Bed    Bed Room            Desk          1
5     Bed Room          Bed    Bed Room            Desk          1
6     Bed Room          Bed    Bed Room            Desk          1
7     Bed Room         Desk    Bed Room            Desk          2
8     Bed Room          Bed    Bed Room            Desk          1
9     Bed Room          Bed    Bed Room            Desk          1


## Activity and Time Processing

In [11]:
# Process activity columns for one-hot encoding
activity_cols = ['activity1', 'activity2', 'activity3']
df_activity = dfContextualFactor[activity_cols].copy()

# Get unique activities across all activity columns, excluding NaN
all_unique_activities = pd.unique(df_activity.values.ravel())
all_unique_activities = [x for x in all_unique_activities if pd.notna(x)]

# Create a DataFrame for one-hot encoding of activities
dfActivity_one_hot_encoding = pd.DataFrame(0, index=df_activity.index, columns=['act_' + str(val) for val in all_unique_activities])

# Perform one-hot encoding for each activity column
for col in activity_cols:
    for val in all_unique_activities:
        dfActivity_one_hot_encoding['act_' + str(val)] |= (df_activity[col] == val).astype(int)

# Concatenate one-hot encoded activity columns to dfCombinedAll
dfActivity_one_hot_encoding
dfCombinedAll = pd.concat([dfCombinedAll, dfActivity_one_hot_encoding], axis=1)

In [12]:
# Convert startTime to datetime and extract total minutes since midnight
dfCombinedAll['startTime'] = pd.to_datetime(dfCombinedAll['startTime'], format='%H:%M:%S', errors='coerce')
dfCombinedAll['minute'] = dfCombinedAll['startTime'].dt.hour * 60 + dfCombinedAll['startTime'].dt.minute

# Map days of the week to numerical values (MON=0, TUE=1, ..., SUN=6)
day_map = {'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6}
dfCombinedAll['dayOfWeek'] = dfCombinedAll['dayOfWeek'].map(day_map)

# Bin minutes into 30-minute intervals for temporal analysis
dfCombinedAll['minute_bin'] = (dfCombinedAll['minute'] // 30).astype(int)
dfCombinedAll.head()

,uid,sid,activity1,activity2,activity3,userRoom,userPosition,weekOfExperiment,dayOfWeek,startTime,...,act_Social Interaction,act_Returning from Outside / Other Rooms,act_Studying / Working,act_Others,act_House Chores,act_Self Caring,act_Visiting Outside / Other Rooms,act_Resting,minute,minute_bin
0,1,1,Taking a Nap / Sleeping,NaN,NaN,Bed Room,Bed,1,0,1900-01-01 09:00:26,...,0,0,0,0,0,0,0,0,540,18
1,1,2,Hygiene,NaN,NaN,Rest Room,NaN,1,0,1900-01-01 10:34:26,...,0,0,0,0,0,0,0,0,634,21
2,1,3,Eating,NaN,NaN,Living Room,NaN,1,1,1900-01-01 09:00:46,...,0,0,0,0,0,0,0,0,540,18
3,1,4,Eating,NaN,NaN,Living Room,NaN,1,1,1900-01-01 09:44:46,...,0,0,0,0,0,0,0,0,584,19
4,1,5,Taking a Nap / Sleeping,NaN,NaN,Bed Room,Bed,1,1,1900-01-01 12:04:48,...,0,0,0,0,0,0,0,0,724,24


In [13]:
dfCombinedAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 39 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   uid                                       2905 non-null   int64         
 1   sid                                       2905 non-null   int64         
 2   activity1                                 2905 non-null   object        
 3   activity2                                 112 non-null    object        
 4   activity3                                 3 non-null      object        
 5   userRoom                                  2905 non-null   object        
 6   userPosition                              2414 non-null   object        
 7   weekOfExperiment                          2905 non-null   int64         
 8   dayOfWeek                                 2905 non-null   int64         
 9   startTime                     

# Feature Extraction

In [18]:
# Select features for response prediction
dfFeatresForResponse = dfCombinedAll[['uid',
    'act_Taking a Nap / Sleeping','act_Hygiene','act_Eating','act_Using Media','act_Social Interaction',
    'act_Returning from Outside / Other Rooms','act_Studying / Working','act_Others','act_House Chores',
    'act_Self Caring','act_Visiting Outside / Other Rooms','act_Resting',
    'homeType','userRoom','userPosition','speakerRoom','speakerPosition',
    'minute_bin','dayOfWeek','SHORT_INTERACTION_availability','LONG_INTERACTION_availability']].copy()

# Encode categorical columns using LabelEncoder
categorical_columns = ['homeType', 'userRoom', 'userPosition', 'speakerRoom', 'speakerPosition', 'minute_bin']
label_encoders = defaultdict(LabelEncoder)

# Apply label encoding to each categorical column
for col in categorical_columns:
    dfFeatresForResponse[col] = label_encoders[col].fit_transform(dfFeatresForResponse[col])

# Create a copy of the encoded data for further processing
encoded_data = dfFeatresForResponse.copy()


encoded_data.info()
encoded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   uid                                       2905 non-null   int64
 1   act_Taking a Nap / Sleeping               2905 non-null   int64
 2   act_Hygiene                               2905 non-null   int64
 3   act_Eating                                2905 non-null   int64
 4   act_Using Media                           2905 non-null   int64
 5   act_Social Interaction                    2905 non-null   int64
 6   act_Returning from Outside / Other Rooms  2905 non-null   int64
 7   act_Studying / Working                    2905 non-null   int64
 8   act_Others                                2905 non-null   int64
 9   act_House Chores                          2905 non-null   int64
 10  act_Self Caring                           2905 non-null   in

,uid,act_Taking a Nap / Sleeping,act_Hygiene,act_Eating,act_Using Media,act_Social Interaction,act_Returning from Outside / Other Rooms,act_Studying / Working,act_Others,act_House Chores,...,act_Resting,homeType,userRoom,userPosition,speakerRoom,speakerPosition,minute_bin,dayOfWeek,SHORT_INTERACTION_availability,LONG_INTERACTION_availability
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,12,0,True,False
1,1,0,1,0,0,0,0,0,0,0,...,0,0,7,3,0,1,15,0,False,False
2,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,12,1,False,False
3,1,0,0,1,0,0,0,0,0,0,...,0,0,5,3,0,1,13,1,False,False
4,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,18,1,True,False


## Label: SHORT_INTERACTION

In [19]:
print(encoded_data['SHORT_INTERACTION_availability'].value_counts())

SHORT_INTERACTION_availability
True     2159
False     746
Name: count, dtype: int64


### Model Building and LOSO CV

In [20]:
# Prepare features (X) and target variables (y) for SHORT_INTERACTION
X = encoded_data.drop(columns=['SHORT_INTERACTION_availability', 'LONG_INTERACTION_availability', 'uid'], axis=1) # Drop target and unrelated columns
y = encoded_data['SHORT_INTERACTION_availability'] # Target variable
groups = encoded_data['uid'] # Group by user ID for Leave-One-Group-Out CV

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models1 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results1 = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)
    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

for model_name, metrics in results1.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2105, number of negative: 2105
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 95
[LightGBM] [Info] Number of data points in the train set: 4210, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2089, number of negative: 2089
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 95
[LightGBM] [Info] Number of data points in the train set: 4178, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is d

...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6542658	total: 1.33ms	remaining: 132ms
1:	learn: 0.6198890	total: 2.45ms	remaining: 120ms
2:	learn: 0.5933189	total: 3.49ms	remaining: 113ms
3:	learn: 0.5698573	total: 4.54ms	remaining: 109ms
4:	learn: 0.5504963	total: 5.46ms	remaining: 104ms
5:	learn: 0.5331454	total: 6.41ms	remaining: 100ms
6:	learn: 0.5172612	total: 7.28ms	remaining: 96.7ms
7:	learn: 0.5019410	total: 8.37ms	remaining: 96.3ms
8:	learn: 0.4888367	total: 9.33ms	remaining: 94.3ms
9:	learn: 0.4751901	total: 10.2ms	remaining: 91.8ms
10:	learn: 0.4667036	total: 11.2ms	remaining: 90.4ms
11:	learn: 0.4554429	total: 12.2ms	remaining: 89.7ms
12:	learn: 0.4471140	total: 13.1ms	remaining: 87.7ms
13:	learn: 0.4396868	total: 14.2ms	remaining: 87.1ms
14:	learn: 0.4320716	total: 15.5ms	remaining: 88ms
15:	learn: 0.4273553	total: 16.9ms	remaining: 88.6ms
16:	learn: 0.4210620	total: 17.8ms	remaining: 87.1ms
17:	learn: 0.4178642	total: 19ms	remaining: 86.6ms
18:	learn: 0.4125859	total: 20ms	remaining: 85.1ms
19:	learn: 0.40

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is d

Random Forest - Accuracy: 0.8321, F1-Score: 0.7373
Gradient Boosting - Accuracy: 0.8128, F1-Score: 0.7082
XGBoost - Accuracy: 0.8209, F1-Score: 0.7183
LightGBM - Accuracy: 0.8229, F1-Score: 0.7148
CatBoost - Accuracy: 0.8250, F1-Score: 0.7276
SVM - Accuracy: 0.7653, F1-Score: 0.6824
Dummy - Accuracy: 0.5064, F1-Score: 0.4413


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1600825135.py:40: FutureWarning: Series.ravel is d

### K-Fold CV

In [21]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results1 = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results1.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1717, number of negative: 1717
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000482 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 3434, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


7:	learn: 0.5047565	total: 7.97ms	remaining: 91.7ms
8:	learn: 0.4926700	total: 9.14ms	remaining: 92.4ms
9:	learn: 0.4837468	total: 10.3ms	remaining: 92.4ms
10:	learn: 0.4752106	total: 11.4ms	remaining: 92.5ms
11:	learn: 0.4658290	total: 12.6ms	remaining: 92.2ms
12:	learn: 0.4574750	total: 13.6ms	remaining: 91ms
13:	learn: 0.4475692	total: 14.6ms	remaining: 89.4ms
14:	learn: 0.4387164	total: 15.6ms	remaining: 88.5ms
15:	learn: 0.4316335	total: 16.6ms	remaining: 86.9ms
16:	learn: 0.4249580	total: 17.6ms	remaining: 85.8ms
17:	learn: 0.4205795	total: 18.7ms	remaining: 85.2ms
18:	learn: 0.4162052	total: 19.8ms	remaining: 84.3ms
19:	learn: 0.4121890	total: 20.9ms	remaining: 83.4ms
20:	learn: 0.4085819	total: 21.9ms	remaining: 82.5ms
21:	learn: 0.4048234	total: 23.1ms	remaining: 82ms
22:	learn: 0.4017888	total: 24.4ms	remaining: 81.8ms
23:	learn: 0.3984529	total: 26ms	remaining: 82.3ms
24:	learn: 0.3962894	total: 27.1ms	remaining: 81.4ms
25:	learn: 0.3941453	total: 28.4ms	remaining: 80.9ms
26

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6579363	total: 1.18ms	remaining: 117ms
1:	learn: 0.6290010	total: 1.98ms	remaining: 96.9ms
2:	learn: 0.6004149	total: 2.85ms	remaining: 92.3ms
3:	learn: 0.5779761	total: 3.8ms	remaining: 91.3ms
4:	learn: 0.5578175	total: 4.69ms	remaining: 89.2ms
5:	learn: 0.5423065	total: 5.54ms	remaining: 86.7ms
6:	learn: 0.5227655	total: 6.38ms	remaining: 84.8ms
7:	learn: 0.5101880	total: 7.3ms	remaining: 83.9ms
8:	learn: 0.4995132	total: 8.21ms	remaining: 83ms
9:	learn: 0.4858153	total: 9.23ms	remaining: 83.1ms
10:	learn: 0.4755228	total: 10.1ms	remaining: 81.8ms
11:	learn: 0.4652177	total: 11ms	remaining: 80.9ms
12:	learn: 0.4562008	total: 12ms	remaining: 80.1ms
13:	learn: 0.4507826	total: 13ms	remaining: 79.7ms
14:	learn: 0.4438381	total: 13.7ms	remaining: 77.7ms
15:	learn: 0.4386896	total: 14.6ms	remaining: 76.6ms
16:	learn: 0.4329875	total: 15.4ms	remaining: 75.3ms
17:	learn: 0.4289006	total: 16.4ms	remaining: 74.8ms
18:	learn: 0.4249671	total: 17.4ms	remaining: 74.1ms
19:	learn: 0.4

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is d


Random Forest - Accuracy: 0.8251, F1-Score: 0.7897
Gradient Boosting - Accuracy: 0.8131, F1-Score: 0.7681
XGBoost - Accuracy: 0.8134, F1-Score: 0.7702
LightGBM - Accuracy: 0.8138, F1-Score: 0.7715
CatBoost - Accuracy: 0.8127, F1-Score: 0.7732
SVM - Accuracy: 0.7711, F1-Score: 0.7345
Dummy - Accuracy: 0.4833, F1-Score: 0.4565


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\1622768231.py:23: FutureWarning: Series.ravel is d

## Label: LONG_INTERACTION

In [22]:
print(encoded_data['LONG_INTERACTION_availability'].value_counts())

LONG_INTERACTION_availability
False    1471
True     1434
Name: count, dtype: int64


### Model building and LOSO CV

In [23]:
# Prepare features (X) and target variable (y) for predicting LONG_INTERACTION
X = encoded_data.drop(columns=['LONG_INTERACTION_availability', 'SHORT_INTERACTION_availability','uid'], axis=1)
y = encoded_data['LONG_INTERACTION_availability']
groups = encoded_data['uid']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models2 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results2 = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results2.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is d

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1428, number of negative: 1428
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 2856, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1367, number of negative: 1367
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000207 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 2734, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


52:	learn: 0.5148309	total: 45.1ms	remaining: 40ms
53:	learn: 0.5142017	total: 46ms	remaining: 39.1ms
54:	learn: 0.5133744	total: 47ms	remaining: 38.4ms
55:	learn: 0.5126256	total: 47.9ms	remaining: 37.6ms
56:	learn: 0.5120342	total: 48.7ms	remaining: 36.7ms
57:	learn: 0.5115081	total: 49.5ms	remaining: 35.8ms
58:	learn: 0.5111041	total: 50.4ms	remaining: 35ms
59:	learn: 0.5105664	total: 51.2ms	remaining: 34.2ms
60:	learn: 0.5098247	total: 52.2ms	remaining: 33.3ms
61:	learn: 0.5094683	total: 53ms	remaining: 32.5ms
62:	learn: 0.5088660	total: 53.9ms	remaining: 31.6ms
63:	learn: 0.5083890	total: 54.7ms	remaining: 30.8ms
64:	learn: 0.5078701	total: 55.7ms	remaining: 30ms
65:	learn: 0.5075740	total: 56.6ms	remaining: 29.2ms
66:	learn: 0.5068480	total: 57.5ms	remaining: 28.3ms
67:	learn: 0.5063955	total: 58.5ms	remaining: 27.5ms
68:	learn: 0.5061943	total: 59.5ms	remaining: 26.7ms
69:	learn: 0.5058279	total: 60.4ms	remaining: 25.9ms
70:	learn: 0.5055142	total: 61.3ms	remaining: 25ms
71:	lea

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is d


Random Forest - Accuracy: 0.6750, F1-Score: 0.6253
Gradient Boosting - Accuracy: 0.6656, F1-Score: 0.5979
XGBoost - Accuracy: 0.6761, F1-Score: 0.6098
LightGBM - Accuracy: 0.6785, F1-Score: 0.6040
CatBoost - Accuracy: 0.6832, F1-Score: 0.6093
SVM - Accuracy: 0.6528, F1-Score: 0.6026
Dummy - Accuracy: 0.4883, F1-Score: 0.4562


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\2371912279.py:40: FutureWarning: Series.ravel is d

### K-Fold CV

In [24]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results2 = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

print()
for model_name, metrics in results2.items():
    print(f'{model_name} - Accuracy: {metrics["Average Accuracy"]:.4f}, F1-Score: {metrics["Average F1-Score (macro)"]:.4f}')

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1191, number of negative: 1191
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 2382, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1203, number of negative: 1203
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000193 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 2406, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6749488	total: 981us	remaining: 97.2ms
1:	learn: 0.6609188	total: 1.92ms	remaining: 94ms
2:	learn: 0.6484952	total: 3.04ms	remaining: 98.4ms
3:	learn: 0.6353003	total: 3.96ms	remaining: 95.1ms
4:	learn: 0.6238569	total: 4.92ms	remaining: 93.5ms
5:	learn: 0.6192477	total: 5.36ms	remaining: 84ms
6:	learn: 0.6114134	total: 6.25ms	remaining: 83ms
7:	learn: 0.6055302	total: 7.27ms	remaining: 83.6ms
8:	learn: 0.5994066	total: 8.2ms	remaining: 82.9ms
9:	learn: 0.5949007	total: 9.13ms	remaining: 82.1ms
10:	learn: 0.5890909	total: 9.98ms	remaining: 80.8ms
11:	learn: 0.5840232	total: 11.1ms	remaining: 81.1ms
12:	learn: 0.5806912	total: 12ms	remaining: 80.6ms
13:	learn: 0.5767384	total: 12.9ms	remaining: 79.3ms
14:	learn: 0.5734428	total: 13.7ms	remaining: 77.9ms
15:	learn: 0.5699424	total: 14.6ms	remaining: 76.4ms
16:	learn: 0.5666100	total: 15.4ms	remaining: 75.3ms
17:	learn: 0.5636922	total: 16.3ms	remaining: 74.2ms
18:	learn: 0.5611353	total: 17.2ms	remaining: 73.3ms
19:	learn: 0.

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is depre


Random Forest - Accuracy: 0.6640, F1-Score: 0.6614
Gradient Boosting - Accuracy: 0.6664, F1-Score: 0.6621
XGBoost - Accuracy: 0.6661, F1-Score: 0.6608
LightGBM - Accuracy: 0.6637, F1-Score: 0.6595
CatBoost - Accuracy: 0.6678, F1-Score: 0.6635
SVM - Accuracy: 0.5966, F1-Score: 0.5951
Dummy - Accuracy: 0.4926, F1-Score: 0.4908


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_23360\715139799.py:23: FutureWarning: Series.ravel is depre